# Imports

In [1]:
from numpy.random import seed
seed(880)
from tensorflow import set_random_seed
set_random_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf
from time import strftime
from PIL import Image

# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGIN_PATH = 'tensorboard_mnist_digit_logs/'

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1
VALIDATION_SIZE = 10000

INPUTS = IMAGE_HEIGHT*IMAGE_WIDTH*IMAGE_CHANNELS

# Get the data

In [4]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

# Data Preprocessing

In [7]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

In [8]:
y_train_all = np.eye(10)[y_train_all]

In [9]:
y_test = np.eye(10)[y_test]
y_test.shape

(10000, 10)

## Validation Dataset from training data

In [10]:
x_val = x_train_all[:10000]
y_val = y_train_all[:10000]

x_train = x_train_all[10000:]
y_train = y_train_all[10000:]


# Setup Tensorflow Graph

In [11]:
X = tf.placeholder(tf.float32, shape=[None,INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, 10], name='labels')

## Neural Network Architecture
### Hyperparameters

In [12]:
nr_epochs = 50
learning_rate = 0.001

n_hidden1 = 512
n_hidden2 = 64

In [13]:
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value = initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
            
        return layer_out

In [14]:
# Model without dropout layer
# layer_1 = setup_layer(X, weight_dim=[INPUTS, n_hidden1],
#                       bias_dim=[n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2],
#                       bias_dim=[n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, 10],
#                       bias_dim=[10], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [15]:
# Model with dropout layer
layer_1 = setup_layer(X, weight_dim=[INPUTS, n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, 10],
                      bias_dim=[10], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [16]:
folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGIN_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Exito')

Exito


# Loss, Optimisation & Metrics

In [17]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

In [18]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

In [19]:
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output,axis=1, name='prediction')
    correct_pred = tf.equal(model_prediction, tf.argmax(Y,axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))

In [20]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

### Display input images

In [21]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [22]:
sess =tf.Session()

## Setup Filewriter and Merge Summaries

In [23]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory+'/validation')

In [24]:
init = tf.global_variables_initializer()
sess.run(init)

## Batching Data

In [25]:
size_of_batch = 1000

In [26]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [27]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

# Training Loop

In [28]:
for epoch in range(nr_epochs):
    
    
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy] , feed_dict=feed_dictionary)
    
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch = {epoch}, Training Accuracy = {batch_accuracy}')
    
    # ================= #
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    
    validation_writer.add_summary(summary, epoch)
    
print('Terminado')

Epoch = 0, Training Accuracy = 0.8629999756813049
Epoch = 1, Training Accuracy = 0.8809999823570251
Epoch = 2, Training Accuracy = 0.8899999856948853
Epoch = 3, Training Accuracy = 0.9670000076293945
Epoch = 4, Training Accuracy = 0.972000002861023
Epoch = 5, Training Accuracy = 0.9700000286102295
Epoch = 6, Training Accuracy = 0.9750000238418579
Epoch = 7, Training Accuracy = 0.9819999933242798
Epoch = 8, Training Accuracy = 0.9810000061988831
Epoch = 9, Training Accuracy = 0.9810000061988831
Epoch = 10, Training Accuracy = 0.9869999885559082
Epoch = 11, Training Accuracy = 0.9860000014305115
Epoch = 12, Training Accuracy = 0.9850000143051147
Epoch = 13, Training Accuracy = 0.9860000014305115
Epoch = 14, Training Accuracy = 0.9860000014305115
Epoch = 15, Training Accuracy = 0.9890000224113464
Epoch = 16, Training Accuracy = 0.9890000224113464
Epoch = 17, Training Accuracy = 0.9909999966621399
Epoch = 18, Training Accuracy = 0.9890000224113464
Epoch = 19, Training Accuracy = 0.98900002

# Saving Model

In [29]:
outputs = {'accuracy_calc/prediction':model_prediction}
inputs = {'X':X}
tf.compat.v1.saved_model.simple_save(sess, 'SavedModel', inputs, outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


# Prediction

In [30]:
img = Image.open('MNIST/test_img.png')

In [31]:
bw = img.convert('L')

In [32]:
img_array = np.invert(bw)

In [33]:
test_img = img_array.ravel()

In [34]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [35]:
print(f'Prediction for test image {prediction}')

Prediction for test image [2]


# Testing and Evaluation

In [1]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
test_accuracy
# print(f'Test accuracy {test_accuracy:0.2%}')

NameError: name 'sess' is not defined

# Reset for the Next Run

In [37]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()

# Pruebas primera parte

In [38]:
# with tf.name_scope('first-hidden-layer'):
#     initial_w1 = tf.truncated_normal(shape=[INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value = initial_w1, name='w1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

In [39]:
# with tf.name_scope('second_hidden_layer'):
#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value = initial_w2, name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [40]:
# with tf.name_scope('output_layer'):
#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, 10], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value = initial_w3, name='w3')

#     initial_b3 = tf.constant(value=0.0, shape=[10])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     layer3_out = tf.nn.softmax(layer3_in)